In [1]:
import os
import sys

# Get the absolute path of the notebook's directory
notebook_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(notebook_dir)

In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from src.app import PermutationSolver


In [3]:
n = 12
config = {
            'n_permutations_length': n,
            
            # Random walks params
            'random_walks_type': 'non-backtracking-beam',
            'n_random_walk_length': int(n * (n-1) / 2),
            'n_random_walks_to_generate': 10000,
            'n_random_walks_steps_back_to_ban': 8,
            
            # Neural Net params
            'model_type': 'MLP',
            'list_layers_sizes': [2**9],
            'n_epochs': 30,
            'batch_size': 1024,
            'lr': 0.001,
            
            # DQN training
            'n_epochs_dqn': 300,
            'flag_dqn_round': False,
            'n_random_walks_to_generate_dqn': 1000,
            'verbose_loc': 5,
            'lr_dqn': 0.000001,
            
            # Beam search
            'beam_search_torch': True,
            'beam_search_Fironov': False,
            'beam_width': 1,
            'n_steps_limit': 4 * n**2,
            'alpha_previous_cost_accumulation': 0,
            'beam_search_models_or_heuristics': 'model_torch',
            'ban_p0_p1_transposition_if_p0_lt_p1_ie_already_sorted': False,
            'n_beam_search_steps_back_to_ban': 32,
            
            # What to solve
            'solve_random_or_longest_state': 'solve_LRX_longest',
            'verbose': 100
        }

In [4]:
n_perm_list = [48]
n_epoch_list = [100]
n_epoch_dqn_list = [300]
batch_size_list = [2**10]
lr_list = [0.000001]
beam_width_list = [2**0]
list_layers_sizes = [[2**13]]

In [5]:
summary_df = pd.DataFrame()

for n_permutations_length in n_perm_list:
    solver = PermutationSolver(config)
    solver.config['n_permutations_length'] = n_permutations_length
    for list_layers_size in list_layers_sizes:
        solver.config['list_layers_sizes'] = list_layers_size
        for lr in lr_list:
            solver.config['lr_dqn'] = lr
            for batch_size in batch_size_list:
                solver.config['batch_size'] = batch_size
                for n_epoch in n_epoch_list:
                    solver.config['n_epochs'] = n_epoch
                    mlp_losses = solver.train_mlp()
                    # save mlp_model
                    torch.save(solver.mlp_model.state_dict(), f'models/mlp_model_{n_permutations_length}_{n_epoch}_{list_layers_sizes[0]}_{lr}_{batch_size}.pth')
                    for n_epoch_dqn in n_epoch_dqn_list:
                        solver.config['n_epochs_dqn'] = n_epoch_dqn
                        dqn_losses = solver.train_dqn()
                        # save dqn_model
                        torch.save(solver.dqn_model.state_dict(), f'models/dqn_model_{n_permutations_length}_{n_epoch_dqn}_{list_layers_sizes[0]}_{lr}_{batch_size}.pth')
                        for beam_width in beam_width_list:
                            solver.config['beam_width'] = beam_width
                            i_step, flag_found_destination = solver.test_beam_search()
                            # save summary_df
                            summary_df = pd.concat([summary_df, pd.DataFrame({'n_permutations_length': n_permutations_length, 'list_layers_sizes': list_layers_sizes, 'lr': lr, 'n_epoch': n_epoch, 'n_epoch_dqn': n_epoch_dqn, 'beam_width': beam_width, 'i_step': i_step, 'flag_found_destination': flag_found_destination, 'mlp_losses': mlp_losses[-1], 'dqn_losses': dqn_losses[-1]})])
                            summary_df.to_csv('models/summary_df.csv', index=False)


Training MLP:  10%|█         | 10/100 [01:19<11:49,  7.89s/it]

Epoch 10, Loss: 47.2713


Training MLP:  20%|██        | 20/100 [02:37<10:30,  7.89s/it]

Epoch 20, Loss: 43.2165


Training MLP:  30%|███       | 30/100 [03:56<09:07,  7.81s/it]

...

Epoch 290 | Loss: 1.2052 | Times - RW: 3.31s, Train: 1.99s | Buffer size: 200000


Training DQN:  99%|█████████▊| 296/300 [27:36<00:22,  5.64s/it]

Epoch 295 | Loss: 1.2379 | Times - RW: 3.41s, Train: 2.39s | Buffer size: 200000


Training DQN: 100%|██████████| 300/300 [27:59<00:00,  5.60s/it]


Training finished in 1679.2s
i_position_X_in_list_generators: 2
Step: 1 Beam (not cumulative) min: 36.14 median: 36.56 max: 37.56
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.001, t_unique_els  0.002, t_full_step 0.010
Step: 11 Beam (not cumulative) min: 35.14 median: 35.28 max: 35.42
Time: 0.0 t_moves  0.001, t_hash  0.000, t_isin 0.001, t_unique_els  0.000, t_full_step 1747165147.164
Step: 21 Beam (not cumulative) min: 34.15 median: 34.66 max: 35.17
Step: 31 Beam (not cumulative) min: 35.05 median: 35.51 max: 35.97
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.000, t_unique_els  0.001, t_full_step 0.002
Step: 41 Beam (not cumulative) min: 34.07 median: 34.52 max: 34.96
Time: 0.0 t_moves  0.000, t_hash  0.001, t_isin 0.000, t_unique_els  0.000, t_full_step 1747165147.217
Step: 51 Beam (not cumulative) min: 33.34 median: 33.77 max: 34.19
Step: 61 Beam (not cumulative) min: 0.00 median: 0.00 max: 0.00
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.001, t_unique_els  0.000, t

In [6]:
solver.config['n_steps_limit'] = 10000

In [7]:
solver.config['beam_width'] = 1024*10
i_step, flag_found_destination = solver.test_beam_search()

i_position_X_in_list_generators: 2
Step: 1 Beam (not cumulative) min: 0.00 median: 0.00 max: 0.00
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.003, t_unique_els  0.001, t_full_step 0.004
Step: 11 Beam (not cumulative) min: 0.00 median: 0.00 max: 0.00
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.002, t_unique_els  0.058, t_full_step 1747165253.029
Step: 21 Beam (not cumulative) min: 31.30 median: 34.37 max: 38.73
Step: 31 Beam (not cumulative) min: 31.01 median: 33.58 max: 38.98
Time: 0.1 t_moves  0.000, t_hash  0.000, t_isin 0.002, t_unique_els  0.054, t_full_step 0.114
Step: 41 Beam (not cumulative) min: 30.41 median: 33.18 max: 40.02
Time: 0.0 t_moves  0.000, t_hash  0.000, t_isin 0.001, t_unique_els  0.051, t_full_step 1747165254.859
Step: 51 Beam (not cumulative) min: 30.30 median: 32.92 max: 38.29
Step: 61 Beam (not cumulative) min: 30.19 median: 32.89 max: 38.69
Time: 0.1 t_moves  0.000, t_hash  0.000, t_isin 0.002, t_unique_els  0.049, t_full_step 0.105
Step: 71 Beam (

In [9]:
# Initialize solver
solver = PermutationSolver(config)

# Train MLP first
print("\nTraining MLP model...")
mlp_losses = solver.train_mlp()

# Train DQN using pre-trained MLP weights
print("\nTraining DQN model with pre-trained MLP weights...")
dqn_losses = solver.train_dqn()

# Test models with beam search
print("\nTesting models with beam search...")
print("\nTesting DQN model...")
dqn_result = solver.test_beam_search()


Training MLP model...


Training MLP:  33%|███▎      | 10/30 [00:05<00:09,  2.04it/s]

Epoch 10, Loss: 1.6647


Training MLP:  47%|████▋     | 14/30 [00:07<00:08,  1.89it/s]


KeyboardInterrupt: 